In [1]:
import json
import pandas as pd

In [2]:
data_path = 'data/doc2dial/'
file_path = data_path + 'doc2dial_dial_train.json'
with open(file_path, 'r') as f:
    data = json.load(f)

# New dataset (only DMV training data)

In [3]:
train_dmv = data['dial_data']['dmv']

train_dmv_df = pd.DataFrame(columns=['question', 'answer', 'ref', 'doc_id', 'dial_id'])
for key in train_dmv.keys():
        #iterate turns
        dial_list = train_dmv[key]
        for dial in dial_list:
            dial_id = dial['dial_id']
            for i in range(0, len(dial['turns']), 2):
            # for role1, role2 in dial['turns']:
                role1 = dial['turns'][i]
                role2 = dial['turns'][i+1] if i+1 < len(dial['turns']) else None
                # make sure role1 is always the user
                if role2 is None:
                    continue
                if role1['role'] == 'user' and role2['role'] == 'agent':
                    question = role1['utterance']
                    answer = role2['utterance']
                    ref = role2['references']
                    train_dmv_df.loc[len(train_dmv_df)] = [question, answer, ref, key, dial_id]
                      
        

In [20]:
len(train_dmv_df)

4150

In [25]:
train_dmv_df.loc[0]

question    Hello, I forgot o update my address, can you h...
answer      hi, you have to report any change of address t...
ref         [{'sp_id': '6', 'label': 'solution'}, {'sp_id'...
doc_id           Top 5 DMV Mistakes and How to Avoid Them#3_0
dial_id                      9f44c1539efe6f7e79b02eb1b413aa43
Name: 0, dtype: object

In [24]:
#write to csv
train_dmv_df.to_csv('data/doc2dial/qa_train_dmv.csv', index=False)

# New Dataset - TEST (only DMV test data)

In [2]:
data_path = 'data/doc2dial/'
file_path = data_path + 'doc2dial_dial_test.json'
with open(file_path, 'r') as f:
    data = json.load(f)

In [3]:
import re

def get_ref(doc, doc2dial_doc):
    refs_ID = re.findall(r"\d+", doc['references'])
    refs_ID = [int(i) for i in refs_ID]
    sp_list = [doc2dial_doc['doc_data'][doc['domain']][doc['doc_id']]['spans'][str(i)] for i in refs_ID]
    return sp_list

In [12]:
test_dmv = data['dial_data']['dmv']
start_agent = 0
start_user = 0

def build(group, turns):
    one_talk = {}
    for i in group:
        if turns[i]['role'] == 'user':
            if 'question' not in one_talk:
                one_talk['question'] = turns[i]['utterance']
            else:
                one_talk['question'] += '##' + turns[i]['utterance']
        else:
            if 'answer' not in one_talk:
                one_talk['answer'] = turns[i]['utterance']
                one_talk['ref'] = [turns[i]['references']]
            else:
                one_talk['answer'] += '##' + turns[i]['utterance']
                one_talk['ref'].append(turns[i]['references'])
    if 'answer' not in one_talk:
        return None
    return one_talk
    
    
    
test_dmv_df = pd.DataFrame(columns=['question', 'answer', 'ref', 'doc_id', 'dial_id'])
for key in test_dmv.keys():
        #iterate turns
        dial_list = test_dmv[key]
        for dial in dial_list:
            dial_id = dial['dial_id']
            # user agent agent | user user agent agent |
            groups = []
            group = []
            for i in range(len(dial['turns'])):
                role = dial['turns'][i]
                if i == len(dial['turns'])-1 or (role['role'] == 'agent' and dial['turns'][i+1]['role'] == 'user'):
                    group.append(i)
                    groups.append(group)
                    group = []
                else:
                    group.append(i)
            for group in groups:
                one_talk = build(group, dial['turns'])
                print(groups)
                if one_talk is not None:
                    test_dmv_df.loc[len(test_dmv_df)] = [one_talk['question'], one_talk['answer'], one_talk['ref'], key, dial_id]
                # else:
                #     print(dial_id)
                #     print(groups)
                #     print(group)
                # print('question:', one_talk['question'])
                # print('answer1:', one_talk['answer'].split('\n')[0])
                # print('answer2:', one_talk['answer'].split('\n')[1] if len(one_talk['answer'].split('\n')) > 1 else '')
                # print('ref:', one_talk['ref'])

[[0, 1, 2], [3, 4, 5], [6, 7, 8], [9, 10], [11, 12], [13, 14]]
[[0, 1, 2], [3, 4, 5], [6, 7, 8], [9, 10], [11, 12], [13, 14]]
[[0, 1, 2], [3, 4, 5], [6, 7, 8], [9, 10], [11, 12], [13, 14]]
[[0, 1, 2], [3, 4, 5], [6, 7, 8], [9, 10], [11, 12], [13, 14]]
[[0, 1, 2], [3, 4, 5], [6, 7, 8], [9, 10], [11, 12], [13, 14]]
[[0, 1, 2], [3, 4, 5], [6, 7, 8], [9, 10], [11, 12], [13, 14]]
[[0, 1, 2], [3, 4, 5], [6, 7], [8, 9], [10, 11]]
[[0, 1, 2], [3, 4, 5], [6, 7], [8, 9], [10, 11]]
[[0, 1, 2], [3, 4, 5], [6, 7], [8, 9], [10, 11]]
[[0, 1, 2], [3, 4, 5], [6, 7], [8, 9], [10, 11]]
[[0, 1, 2], [3, 4, 5], [6, 7], [8, 9], [10, 11]]
[[0, 1], [2, 3], [4, 5, 6], [7, 8, 9], [10, 11]]
[[0, 1], [2, 3], [4, 5, 6], [7, 8, 9], [10, 11]]
[[0, 1], [2, 3], [4, 5, 6], [7, 8, 9], [10, 11]]
[[0, 1], [2, 3], [4, 5, 6], [7, 8, 9], [10, 11]]
[[0, 1], [2, 3], [4, 5, 6], [7, 8, 9], [10, 11]]
[[0, 1], [2, 3], [4, 5, 6], [7, 8, 9], [10, 11], [12, 13], [14, 15]]
[[0, 1], [2, 3], [4, 5, 6], [7, 8, 9], [10, 11], [12, 13], [14,

In [5]:
len(test_dmv_df)

1006

In [8]:
test_dmv_df.loc[1]['question']

'What if iI already moved?##Please go to a local branch with a piece of mail and update your address.'

In [9]:
test_dmv_df.loc[1]['answer']

'Please go to a local branch and update your address.'

In [30]:
# write to csv
test_dmv_df.to_csv('data/doc2dial/qa_test_dmv.csv', index=False)

# END

In [25]:
domains = ['dmv', 'ssa', 'va', 'studentaid']

def run():
    df = pd.DataFrame(columns=['question', 'answer', 'domain', 'doc_id', 'references', 'dial_id'])
    cnt = 0
    for domain in domains:
        sub_dict = data['dial_data'][domain]
        #iterate sub_dict
        for key in sub_dict.keys():
            #iterate turns
            dial_list = sub_dict[key]
            for dial in dial_list:
                dial_id = dial['dial_id']
                for turn in dial['turns']:
                    role = turn['role']
                    ref = turn['references']

                    if role == 'user':
                        question =  turn['utterance']
                    elif role == 'agent':
                        answer =  turn['utterance']
                        df.loc[len(df)] = [question, answer, domain, key, ref, dial_id]
                        # qa_list.append([question, answer, domain, ref])
                        cnt += 1
                    else:
                        print('Error: No such role')
                        
    print('Total QA pairs: ', cnt)
    df.to_csv(data_path + 'doc2dial_qa.csv', index=False)           
            


In [26]:
run()

Total QA pairs:  21998


In [22]:
df = pd.read_csv(data_path + 'doc2dial_qa_train.csv')
df.head(10)

,question,answer,domain,doc_id,references,dial_id
0,"Hello, I forgot o update my address, can you h...","hi, you have to report any change of address t...",dmv,Top 5 DMV Mistakes and How to Avoid Them#3_0,"[{'sp_id': '6', 'label': 'solution'}, {'sp_id'...",9f44c1539efe6f7e79b02eb1b413aa43
1,Can I do my DMV transactions online?,"Yes, you can sign up for MyDMV for all the onl...",dmv,Top 5 DMV Mistakes and How to Avoid Them#3_0,"[{'sp_id': '56', 'label': 'solution'}]",9f44c1539efe6f7e79b02eb1b413aa43
2,"Thanks, and in case I forget to bring all of t...",This happens often with our customers so that'...,dmv,Top 5 DMV Mistakes and How to Avoid Them#3_0,"[{'sp_id': '49', 'label': 'solution'}, {'sp_id...",9f44c1539efe6f7e79b02eb1b413aa43
3,"Ok, and can you tell me again where should I r...",Sure. Any change of address must be reported t...,dmv,Top 5 DMV Mistakes and How to Avoid Them#3_0,"[{'sp_id': '6', 'label': 'solution'}, {'sp_id'...",9f44c1539efe6f7e79b02eb1b413aa43
4,Can you tell me more about Traffic points and ...,Traffic points is the system used by DMV to tr...,dmv,Top 5 DMV Mistakes and How to Avoid Them#3_0,"[{'sp_id': '41', 'label': 'solution'}, {'sp_id...",9f44c1539efe6f7e79b02eb1b413aa43
5,Will you keep me informed about when my insura...,Yes. The New York State requires every motoris...,dmv,Top 5 DMV Mistakes and How to Avoid Them#3_0,"[{'sp_id': '24', 'label': 'solution'}, {'sp_id...",88ecf840ea87f8c53faff15d4f0bb214
6,Is it needed that my address is always up to d...,"Yes, It is required. If not we won't be able t...",dmv,Top 5 DMV Mistakes and How to Avoid Them#3_0,"[{'sp_id': '35', 'label': 'solution'}, {'sp_id...",88ecf840ea87f8c53faff15d4f0bb214
7,Can you tell me more about the traffic points ...,Yes. This is the system DM uses to keep a reco...,dmv,Top 5 DMV Mistakes and How to Avoid Them#3_0,"[{'sp_id': '41', 'label': 'solution'}, {'sp_id...",88ecf840ea87f8c53faff15d4f0bb214
8,It's always a hassle to bring all the required...,This happens quite often with our customers wh...,dmv,Top 5 DMV Mistakes and How to Avoid Them#3_0,"[{'sp_id': '49', 'label': 'solution'}, {'sp_id...",88ecf840ea87f8c53faff15d4f0bb214
9,In case I have a problem with my insurance and...,"In that case, we must suspend the vehicle regi...",dmv,Top 5 DMV Mistakes and How to Avoid Them#3_0,"[{'sp_id': '32', 'label': 'solution'}, {'sp_id...",88ecf840ea87f8c53faff15d4f0bb214


In [23]:
len(df)

21998

In [34]:
#mapping doc_id to dail_id
df[['doc_id', 'dial_id']]
#remove duplicates
df = df.drop_duplicates(subset=['doc_id', 'dial_id'])

3474

In [35]:
len(df[['doc_id', 'dial_id']])

3474

In [42]:
df[['dial_id', 'doc_id']].head(10)

,dial_id,doc_id
0,9f44c1539efe6f7e79b02eb1b413aa43,Top 5 DMV Mistakes and How to Avoid Them#3_0
5,88ecf840ea87f8c53faff15d4f0bb214,Top 5 DMV Mistakes and How to Avoid Them#3_0
10,3c079944cf36c05b45f668669cc3301b,Top 5 DMV Mistakes and How to Avoid Them#3_0
15,9fa8c68e2f211edad9fb65eb5897d600,Top 5 DMV Mistakes and How to Avoid Them#3_0
20,423bd6a66d14c22f8be69add96679255,Top 5 DMV Mistakes and How to Avoid Them#3_0
25,15aa88cc47d6c698af75ce5a00cc7c04,Top 5 DMV Mistakes and How to Avoid Them#3_0
31,eec4a276b8f0aebb4069d0dbca94b30f,Top 5 DMV Mistakes and How to Avoid Them#3_0
37,8b326c47087dfbec10f9ad1e4df1df2b,Top 5 DMV Mistakes and How to Avoid Them#3_0
42,494fe90938a0d159620d0229fbe059b2,Top 5 DMV Mistakes and How to Avoid Them#3_0
49,1f84f158224bcc4b92bf466b7f0986b7,Top 5 DMV Mistakes and How to Avoid Them#3_0


In [46]:
df[['dial_id', 'doc_id']]
#build a dictionary mapping dial_id to doc_id
dial2doc = {}
for index, row in df.iterrows():
    dial2doc[row['dial_id']] = row['doc_id']
    
#build a dictionary mapping doc_id to dial_id (one to many)
doc2dial = {}
for index, row in df.iterrows():
    if row['doc_id'] in doc2dial:
        doc2dial[row['doc_id']].append(row['dial_id'])
    else:
        doc2dial[row['doc_id']] = [row['dial_id']]


In [52]:
#write dial2doc and doc2dial to json file
output = {'dial2doc':dial2doc, 'doc2dial':doc2dial}
with open(data_path + 'doc2dial_mapping_train.json', 'w') as f:
    json.dump(output, f)

In [2]:
#load qa from csv file
data_path = 'data/doc2dial/'
df = pd.read_csv(data_path + 'doc2dial_qa_train.csv')

In [3]:
#remove duplicates
df = df.drop_duplicates(subset=['doc_id'])

In [4]:
df

,question,answer,domain,doc_id,references,dial_id
0,"Hello, I forgot o update my address, can you h...","hi, you have to report any change of address t...",dmv,Top 5 DMV Mistakes and How to Avoid Them#3_0,"[{'sp_id': '6', 'label': 'solution'}, {'sp_id'...",9f44c1539efe6f7e79b02eb1b413aa43
70,Can I know if anybody asked for my records?,"Yes, since the records of searches are public ...",dmv,DIAL-IN search accounts#3_0,"[{'sp_id': '60', 'label': 'solution'}]",6ad62ea7a0a74030c0df7cd5be594e1f
151,Can I use insurance documents from other states?,We can't accept any out-of-state vehicle insur...,dmv,New York State Insurance Requirements#3_0,"[{'sp_id': '46', 'label': 'solution'}]",b9d3da1f4aba75ee4729d8f91030e16f
215,Tell me about county taxes and supplemental fees,This are taxes that all residents of New York ...,dmv,"Commercial vehicle registration fees, vehicle ...","[{'sp_id': '14', 'label': 'solution'}, {'sp_id...",14bf00331123b8ad7ddef90f7de8b081
279,Can you tell me when do I need to surrender my...,You should surrender your vehicle's plate befo...,dmv,Surrender (return or turn in) your plates to t...,"[{'sp_id': '2', 'label': 'solution'}]",7b12a88d3fefc83153d143495691f35b
...,...,...,...,...,...,...
21715,What kind of counseling is required before I r...,Have you heard of entrance counseling?,studentaid,Complete Federal Student Loan Counseling | Fed...,"[{'sp_id': '3', 'label': 'precondition'}]",75908a260f83e4b2c7806f6f0122d851
21769,I am in need of lowering or suspending my stud...,We can help you stay on track to be sucessful ...,studentaid,Lower or Suspend Student Loan Payments | Feder...,"[{'sp_id': '2', 'label': 'solution'}]",f151402888e8779680628e40530aef14
21821,Could you help me understand how to repay my f...,First you should understand your loan agreemen...,studentaid,Avoiding Default | Federal Student Aid#1_0,"[{'sp_id': '2', 'label': 'solution'}, {'sp_id'...",eb098ae8af4c436cef462cd5960590a1
21872,How would I go about applying for a Total and ...,You would need to complete a TPD discharge app...,studentaid,Total and Permanent Disability Discharge | Fed...,"[{'sp_id': '36', 'label': 'solution'}, {'sp_id...",1477373287bc5511014cbc8c14da7862


In [5]:
df[['doc_id', 'domain']].head(5)

,doc_id,domain
0,Top 5 DMV Mistakes and How to Avoid Them#3_0,dmv
70,DIAL-IN search accounts#3_0,dmv
151,New York State Insurance Requirements#3_0,dmv
215,"Commercial vehicle registration fees, vehicle ...",dmv
279,Surrender (return or turn in) your plates to t...,dmv


In [6]:
#load json file
file_path = 'data/doc2dial/doc2dial_doc.json'
with open(file_path, 'r') as f:
    doc2dial_doc = json.load(f)

In [7]:
doc2dial_doc['doc_data'].keys()

dict_keys(['ssa', 'va', 'dmv', 'studentaid'])

In [9]:
doc2dial_doc['doc_data']['dmv']['DIAL-IN search accounts#3_0']

{'title': 'DIAL-IN search accounts#3',
 'doc_id': 'DIAL-IN search accounts#3_0',
 'domain': 'dmv',
 'doc_text': "Log On to DIAL - IN [1 ] \n\nWhat is a DIAL-IN search account and who can get one? \nIf you regularly need to search DMV records for driver license, registration, vehicle title or insurance information , then DIAL - IN is the service you need. Our typical clients include attorneys, insurance and private investigators, process servers and building / site security personnel. See Drivers Privacy Protection Act DPPA [2] for more information about permissible uses of DMV records. \n\nHow do I apply for an account? \nYou should complete form Motor Vehicle Record Search Account Application PDF [3] MV-15D. The instructions are on the form. Send the form to : Data Services - New Search AccountNYS Department of Motor Vehicles6 Empire State PlazaAlbany, NY 12228 \n\nHow will I know my application has been accepted? \nWe will review your application. If you application is not complete ,

In [25]:
#loop through each row
for index, row in df.iterrows():
    doc_id = row['doc_id']
    domain = row['domain']
    doc_text = doc2dial_doc['doc_data'][domain][doc_id]['doc_text']
    sp = doc2dial_doc['doc_data'][domain][doc_id]['spans']
    doc_title = doc2dial_doc['doc_data'][domain][doc_id]['doc_title']
    

Top 5 DMV Mistakes and How to Avoid Them#3_0 dmv
DIAL-IN search accounts#3_0 dmv
New York State Insurance Requirements#3_0 dmv
Commercial vehicle registration fees, vehicle use taxes and supplemental fees#3_0 dmv
Surrender (return or turn in) your plates to the DMV#3_0 dmv
Renew non-driver ID card#1_0 dmv
Refunds and transfer credits for surrendered plates#3_0 dmv
Prepare for your road test#3_0 dmv
Appeal a TVB ticket conviction#1_0 dmv
Registrations#3_0 dmv
Get a motorcycle license#1_0 dmv
Register a vehicle I bought at a sheriff or marshal auction#3_0 dmv
Replace license or permit#1_0 dmv
Medical Certification FAQ's#3_0 dmv
About the NYS Driver Point System#3_0 dmv
Surrender IRP registration and plates#1_0 dmv
Replace a title certificate#1_0 dmv
Escort driver certification#1_0 dmv
Stolen and recovered vehicles#3_0 dmv
About complaints and charges for repair and body shops#3_0 dmv
File a Freedom of Information Law (FOIL) request#1_0 dmv
Register a moped#1_0 dmv
Schedule and take a roa

In [2]:
path = 'data/doc2dial/qa_train_dmv.csv'
df = pd.read_csv(path)

In [3]:
len(df.dropna())

4139

In [16]:
train_df = df.dropna()[:1000]

In [17]:
test_df = df.dropna()[3000:3800]

In [19]:
len(train_df)

1000

In [15]:
len(test_df.dropna())

798

In [8]:
from sklearn.model_selection import train_test_split

In [12]:
train_df, test_df = train_test_split(df.dropna(), test_size=0.2, random_state=42)

In [13]:
len(train_df)

3311

In [15]:
len(test_df)

828

In [11]:
train_df.to_csv('data/doc2dial/new_dataset/train_set_norandom.csv', index=False)
test_df.to_csv('data/doc2dial/new_dataset/test_set_norandom.csv', index=False)

In [18]:
test = pd.read_csv('data/doc2dial/new_dataset/test_set.csv')